<a href="https://colab.research.google.com/github/MaikeFogaca/Python-Projects/blob/main/scraping_OLX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Daremos início ao nosso estudo gerando um modelo de Machine Learning cujo objetivo é prever o valor do aluguel de imóveis localizados na cidade de Ponta Grossa PR.

Análise Exploratória de Dados

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from time import sleep
import re
import numpy as np

In [ ]:
re_precos = '<span class="price__fraction">(.*)</span>'
re_quartos = r'\| (.*) quarto'
re_areas = '> (.*) m² '
re_quarto_area_ausente = "> (.*) quarto"

In [ ]:
zonas = ['norte', 'sul', 'leste', 'oeste']
paginas = ['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193', 
           '_Desde_241', '_Desde_289']
dados = {'zona': [],
         'quartos': [],
         'area': [],
         'preco': []}

for zona in zonas:
  for pagina in paginas:
    url = f'https://imoveis.mercadolivre.com.br/apartamentos/aluguel/parana/ponta-grossa/'

    c = requests.get(url).content
    soup = BeautifulSoup(c, 'html.parser')

    preco = soup.find_all(name='span', attrs={'class': 'price__fraction'})
    atributos = soup.find_all(name='div', attrs={'class': 'item__attrs'})

    for p in preco:
      valor = re.findall(re_precos, str(p))[0]
      dados['preco'].append(valor)
    
    for atributo in atributos:
      n_quartos = re.findall(re_quartos, str(atributo))
      area = re.findall(re_areas, str(atributo))

      if n_quartos == []:
        n_quartos = re.findall(re_quarto_area_ausente, str(atributo))
        if n_quartos == []:
          n_quartos = np.nan
        else:
          n_quartos = n_quartos[0]
      else:
        n_quartos = n_quartos[0]
      dados['quartos'].append(n_quartos)

      if area == []:
        area = np.nan
      else:
        area = area[0]      
      dados['area'].append(area)

      dados['zona'].append(zona)
    sleep(5)

In [ ]:
df = pd.DataFrame(dados)

In [ ]:
df


,zona,quartos,area,preco


In [ ]:
df['preco'] = df['preco'].map(lambda x: x.replace('.', ''))

In [ ]:
df.head(10)


,zona,quartos,area,preco


In [ ]:
df.to_csv('dados_imoveis.csv')

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
from time import sleep
import pandas as pd

zonas = ['norte', 'sul', 'leste', 'oeste']
paginas = ['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193', '_Desde_241', '_Desde_289']
url_ml = 'https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-{}/{}'

re_precos = r'<span class="price-tag-fraction">(.*)<'
re_quartos = r'([1-9]{1,2}) quarto[s]?'
re_areas = r'<li class="ui-search-card-attributes__attribute">(.*) m²'


class Scraper:
    zonas = []
    areas = []
    quartos = []
    precos = []
    dados = {}

    def __init__(self, url, zona, pagina):

        self.zona = zona
        self.url = url.format(zona, pagina)

    def get_atributes(self):

        c = requests.get(self.url).content
        soup = BeautifulSoup(c, 'html.parser')

        for dado in soup.select('.ui-search-result__content-wrapper'):
            preco = dado.select_one('.price-tag-fraction')
            metros = dado.select_one('.ui-search-card-attributes__attribute ')

            precos = re.findall(re_precos, str(preco))
            areas = re.findall(re_areas, str(metros))
            quartos = re.findall(re_quartos, dado.text)

            for p in precos:
                if isinstance(p, str):
                    p = int(p.replace('.', '').replace(',', '').strip())
                    self.precos.append(p)

            if len(areas) == 0:
                areas.append(np.nan)
            for a in areas:
                if isinstance(a, str):
                    a = int(a.replace('.', '').replace(',', '').strip())
                self.areas.append(a)

            if len(quartos) == 0:
                quartos.append(np.nan)
            for q in quartos:
                self.quartos.append(q)

            self.zonas.append(self.zona)

            self.dados['zona'] = self.zonas
            self.dados['quartos'] = self.quartos
            self.dados['area'] = self.areas
            self.dados['preco'] = self.precos

        sleep(2)
        return self.dados

    def create_csv(self):
        df = pd.DataFrame(self.dados)
        df.to_csv('dados_imoveis.csv', index=False)


if __name__ == '__main__':
    for zona in zonas:
        for pagina in paginas:
            sleep(2)
            scrap = Scraper(url_ml, zona, pagina)
            scrap.get_atributes()
            scrap.create_csv()

KeyboardInterrupt: ignored